### Install Modules

In [1]:
# !pip install pydicom
# !pip install matplotlib
# !pip install seaborn
# !pip install opencv_python
# !pip install tqdm
# !pip install scikit-image
# !pip install nibabel
# !pip install scikit-image
# !pip install gdcm
# !pip install pylibjpeg
# !pip install pylibjpeg pylibjpeg-libjpeg pydicom[gdcm]
# !pip install "dask[dataframe]"
# !pip install imageio

### Import Dependencies

In [1]:
import numpy as np
import pandas as pd
# import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.patches as patches
import seaborn as sns
sns.set(style='darkgrid', font_scale=1.6)
import cv2
import os
from os import listdir
import re
import gc
import pydicom
import datetime
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import nibabel as nib
from glob import glob
import dask.array as da
import warnings

### Load Segmentation file

In [16]:
def load_NIfTI(path):
    mask = nib.load(path)
    
    # Convert to numpy array
    seg = mask.get_fdata()
    
    # Align orientation with images
    seg = seg[:, ::-1, ::-1].transpose(2, 1, 0)
    
    return seg

In [17]:
#Getting patient with mask
seg_paths = glob(f'G:/Data/Spine_Data/segmentations/*')
training_patient=[]
for path in seg_paths:
    training_patient.append((path.rsplit("\\",1)[-1])[:-4])#Patient with mask present

In [18]:
#Example segment image
path_mask=f"G:/Data/Spine_Data/segmentations/{training_patient[-7]}.nii"
patient_mask=load_NIfTI(path_mask)

patient_mask.shape

(195, 512, 512)

### Apply windowing to the image

In [26]:
def apply_windowing(image, window_width, window_level):
    # Apply windowing to the image
    min_value = window_level - (window_width // 2)
    max_value = window_level + (window_width // 2)
    windowed_image = np.clip(image, min_value, max_value)
    windowed_image = (windowed_image - min_value) / (max_value - min_value)
    windowed_image = (windowed_image * 255).astype(np.uint8)
    return windowed_image

# ...

# Inside the loop

# Apply windowing to the image slice range
# image_slice_range = apply_windowing(image_slice_range, window_width, window_level)

## Creat bondingbox from Segmentation Mask

In [27]:
import os
import numpy as np
import nibabel as nib
import pydicom
from skimage.measure import label, regionprops
import imageio

start = 0

# Create the output directories for saving images and bounding boxes if they don't exist
images_directory = "images"
os.makedirs(images_directory, exist_ok=True)
bounding_boxes_directory = "labels"
os.makedirs(bounding_boxes_directory, exist_ok=True)

for i in range(start, start + 40):
    patient_ID = training_patient[i]
    
    # Load segmentation mask
    mask = load_NIfTI(f"G:/Data/Spine_Data/segmentations/{patient_ID}.nii")
    
    # Select specific slices from the mask
    mask_slice_range = mask[:]
    
    # Get unique component labels in the mask
    component_labels = np.unique(mask_slice_range)
    
    # Iterate over each component label
    for label_value in component_labels:
        # Exclude background label (0)
        if label_value == 0:
            continue
        
        # Assign class based on label value
        obj_class = int(label_value)
        
        # Skip labels greater than 7
        if obj_class > 7:
            continue
        
        # Create a binary mask for the current label
        binary_mask = (mask_slice_range == label_value).astype(np.uint8)
        
        # Label connected components in the binary mask
        labeled_mask = label(binary_mask)
        
        # Find the components with the current label
        props = regionprops(labeled_mask)
        
        # Load patient scan
        patient_scan = load_scan(f"G:/Data/Spine_Data/train_images/{patient_ID}")
        
        # Select specific slices from the scan
        image_slice_range = get_pixels_hu(patient_scan)[:]
        
        # Apply windowing to the image slice range
        window_width = 2000
        window_level = 500
        image_slice_range = apply_windowing(image_slice_range, window_width, window_level)
        
        # Save each image slice as PNG and corresponding bounding boxes
        for idx, image_slice in enumerate(image_slice_range):
            current_slice_binary_mask = binary_mask[idx]
            
            # Check if the binary mask has non-zero values
            if np.count_nonzero(current_slice_binary_mask) == 0:
                continue
            
            image_filename = f"{patient_ID}_{idx}.png"
            image_filepath = os.path.join(images_directory, image_filename)  # Update the desired output directory
            imageio.imwrite(image_filepath, image_slice)
            
            # Calculate bounding box coordinates for each object in the current image slice
            labeled_objects = label(current_slice_binary_mask)
            object_labels = np.unique(labeled_objects)
            
            for object_label in object_labels:
                if object_label == 0:
                    continue
                
                object_mask = (labeled_objects == object_label).astype(np.uint8)
                non_zero_pixels = np.nonzero(object_mask)
                
                if non_zero_pixels[0].size == 0:
                    continue
                
#                 min_col = np.min(non_zero_pixels[1])
#                 min_row = np.min(non_zero_pixels[0])
#                 max_col = np.max(non_zero_pixels[1])
#                 max_row = np.max(non_zero_pixels[0])
                margin = 10
                min_col = np.min(non_zero_pixels[1])
                min_row = np.min(non_zero_pixels[0])
                max_col = np.max(non_zero_pixels[1]) + margin
                max_row = np.max(non_zero_pixels[0]) + margin

                # Ensure the bounding box coordinates are within the image dimensions
                min_col = max(0, min_col)
                min_row = max(0, min_row)
                max_col = min(image_slice.shape[1] - 1, max_col)
                max_row = min(image_slice.shape[0] - 1, max_row)

                # Skip bounding boxes with class values greater than 7
                if obj_class > 7:
                    continue
                
                # Save bounding box coordinates in TXT file
                txt_filename = f"{patient_ID}_{idx}.txt"
                txt_filepath = os.path.join(bounding_boxes_directory, txt_filename)  # Update the desired output directory

                
                with open(txt_filepath, "w") as txt_file:
                    txt_file.write(f"{obj_class} {min_col} {min_row} {max_col} {max_row}")

### Normalized Bondingbox

In [13]:
# import os

# # Folder path containing the label files
# labels_folder = "labels"

# # Image dimensions
# image_width = 512  # Width of the image in pixels
# image_height = 512  # Height of the image in pixels

# # Iterate over the label files in the folder
# for filename in os.listdir(labels_folder):
#     if filename.endswith(".txt"):
#         label_file = os.path.join(labels_folder, filename)

#         # Read the label file
#         with open(label_file, "r") as file:
#             lines = file.readlines()

#         # Normalize the bounding box coordinates
#         normalized_lines = []
#         for line in lines:
#             parts = line.strip().split(" ")
#             class_label = parts[0]
#             x_min, y_min, width, height = map(float, parts[1:])

#             # Normalize the bounding box coordinates
#             normalized_x_min = x_min / image_width
#             normalized_y_min = y_min / image_height
#             normalized_width = width / image_width
#             normalized_height = height / image_height

#             # Append the normalized coordinates with the class information to the new list
#             normalized_lines.append(f"{class_label} {normalized_x_min} {normalized_y_min} {normalized_width} {normalized_height}\n")

#         # Save the normalized bounding box coordinates back to the label file
#         with open(label_file, "w") as file:
#             file.writelines(normalized_lines)



### Convert class from 1 to 7 to 0 to 6

In [2]:
import os

# Folder path containing the label files
# labels_folder = "F:/PythonEn/Code/Datav8/val/labels"

# Iterate over the label files in the folder
for filename in os.listdir(labels_folder):
    if filename.endswith(".txt"):
        label_file = os.path.join(labels_folder, filename)

        # Read the label file
        with open(label_file, "r") as file:
            lines = file.readlines()

        # Convert the class labels
        converted_lines = []
        for line in lines:
            parts = line.strip().split(" ")
            class_label = int(parts[0]) - 1  # Convert class from 1 to 7 to 0 to 6

            # Append the converted class label with the remaining parts to the new list
            converted_lines.append(f"{class_label} {' '.join(parts[1:])}\n")

        # Save the converted lines back to the label file
        with open(label_file, "w") as file:
            file.writelines(converted_lines)

### Yolo v5 & v8

In [ ]:
import git

In [2]:
import torch

In [3]:
from IPython.display import Image

In [6]:
# !git clone https://github.com/ultralytics/yolov5

In [1]:
# !cd yolov5 & pip install -r requirements.txt

In [10]:
from IPython.display import Image

In [2]:
# import os

# # Change to the directory where requirements.txt is located
# os.chdir("F:\PythonEn\Code\yolov5")

# # Install the requirements
# !pip install -r requirements.txt

### Data Preparation Yolov5 Format

In [37]:
# Divide the dataset in train and val folder.
import os
from random import choice
import shutil

#arrays to store file names
imgs =[]
xmls =[]

#setup dir names
trainPath = 'F:/PythonEn/Code/yolov5/dataset/images/train'
valPath = 'F:/PythonEn/Code/yolov5/dataset/images/val'
crsPath = 'F:/PythonEn/Code/yolov5/image/' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)

training images are :  7337
Validation images are :  1834


In [38]:
trainimagePath = 'F:/PythonEn/Code/yolov5/dataset/images/train'
trainlabelPath = 'F:/PythonEn/Code/yolov5/dataset/labels/train'
valimagePath = 'F:/PythonEn/Code/yolov5/dataset/images/val'
vallabelPath = 'F:/PythonEn/Code/yolov5/dataset/labels/val'
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)



#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)
shutil.move(crsPath, valPath)

'F:/PythonEn/Code/yolov5/dataset/images/val\\image'

### Yolov5

In [23]:
# !pip uninstall yolov5
# !pip install yolov5

In [1]:
import torch

In [ ]:
!cd yolov5 && python train.py --img 512 --batch 4 --epochs 3 --data custom_data.yaml --weights yolov5m.pt --workers 2

### Yolov8

In [22]:
!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20  Python-3.8.6 torch-1.8.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
Setup complete  (12 CPUs, 15.8 GB RAM, 65.6/431.5 GB disk)


In [24]:
!pip install ultralytics==8.0.20

In [23]:
from ultralytics import YOLO

from IPython.display import display, Image

In [28]:
!yolo task=detect mode=train model=yolov8m.pt data=custom.yaml epochs=200 imgsz=512


  0%|          | 0.00/49.7M [00:00<?, ?B/s]
  1%|1         | 536k/49.7M [00:00<00:10, 5.02MB/s]
  2%|2         | 1.01M/49.7M [00:00<00:10, 4.82MB/s]
  3%|3         | 1.54M/49.7M [00:00<00:10, 4.95MB/s]
  5%|4         | 2.41M/49.7M [00:00<00:07, 6.28MB/s]
  7%|7         | 3.59M/49.7M [00:00<00:05, 8.22MB/s]
  9%|9         | 4.60M/49.7M [00:00<00:05, 8.72MB/s]
 11%|#         | 5.44M/49.7M [00:00<00:05, 8.49MB/s]
 13%|#2        | 6.33M/49.7M [00:00<00:05, 8.50MB/s]
 14%|#4        | 7.14M/49.7M [00:01<00:09, 4.70MB/s]
 16%|#5        | 7.77M/49.7M [00:01<00:10, 4.14MB/s]
 17%|#6        | 8.30M/49.7M [00:01<00:10, 4.31MB/s]
 18%|#7        | 8.83M/49.7M [00:01<00:09, 4.48MB/s]
 19%|#8        | 9.34M/49.7M [00:01<00:09, 4.57MB/s]
 20%|##        | 10.0M/49.7M [00:01<00:08, 5.06MB/s]
 22%|##1       | 10.8M/49.7M [00:02<00:06, 5.87MB/s]
 23%|##3       | 11.6M/49.7M [00:02<00:06, 6.55MB/s]
 25%|##5       | 12.6M/49.7M [00:02<00:05, 7.38MB/s]
 28%|##7       | 13.8M/49.7M [00:02<00:04, 8.42MB/s]
 3